In [159]:
#Load datafile and import packages

fdir = "/home/julian/repositories/RTK_on_the_beach/hanglog/soper"

import os, sys, datetime
sys.path.append("..")
import hacktrack.loaders
import pandas, numpy, math
from scipy import optimize
from hacktrack import loaders, utils

logfiles = os.listdir(fdir)
print(logfiles)
fLog = os.path.join(fdir, logfiles[0])
fd = loaders.FlyDat(fLog)

rectypespresent = [k for k in fd.reccounts  if fd.reccounts[k] != 0]
fd.LoadC("".join(rectypespresent))

['hdata-2021-03-28_10-48-33.log', 'hdata-2021-03-30_16-38-35.log', 'hdata-2021-03-30_15-32-49.log', 'hdata-2021-03-30_15-12-18.log', 'hdata-2021-03-28_22-36-06.log', 'hdata-2021-03-28_22-05-18.log', 'hdata-2021-03-30_15-29-08.log', 'hdata-2021-03-30_17-03-31.log']
Extracting knowndate from filename: 2021-03-28
Error: Should be same midnight stamps 2021-03-28 00:00:00 2021-03-27 00:00:00
End found End(8811,3247)
Z:8142, U:29, S:345, X:295, aZ:2485, aQ:72, aF:617, aV:72
linAdifftime 48100760.63192183 count 614
Made for Q 0 last index 0
Made for V 0 last index 0
Loading aQV as no pQV records
Made for aQ 72 last index 72
pCattrname aQ
Setting aRdatetime0 2021-03-28 20:24:23.918916666 from 2021-03-27 21:24:23.963000 with std 14.95
Made for aV 72 last index 72
pCattrname aV
BAD VALUES 8142 [(0, 'Zt000244AAx0000yFFF9zFFFEaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0000n0008oFFF7\n'), (0, 'Zt000244B5xFFFFyFFFBzFFFBaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0002n0002o0000\n'), (0, 'Zt000244C1xFFF6y0002z

7

In [153]:
os.path.split(fd.fname)[1]

'hdata-2021-03-28_10-48-33.log'

In [145]:
try:
    influxdbconfig = dict(l.split()  for l in open("../influxdb_password.txt").readlines()  if l.strip())
except FileNotFoundError:
    print("Missing password file, creating blank one to fill in")
    fpass = open("../influxdb_password.txt", "w")
    fpass.write("influxdbuser      hanglog\n")
    fpass.write("influxdbpassword      password-goes-here\n")
    fpass.close()
    influxdbconfig["influxdbpassword"] = "password-goes-here"
if influxdbconfig["influxdbpassword"] == "password-goes-here":
    print("please set the password in the file influxdb_password.txt")
    #sys.exit()



In [146]:
import influxdb
client = influxdb.InfluxDBClient(host='influxdb.doesliverpool.xyz', port=8086, 
                                 username=influxdbconfig["influxdbuser"], password=influxdbconfig["influxdbpassword"])
print("databases present", client.get_list_database())
client.switch_database('hanglogdb')

databases present [{'name': 'ming_default'}, {'name': '_internal'}, {'name': 'julianhousedb'}, {'name': 'rosebankdb'}, {'name': 'hanglogdb'}]


In [74]:
tags = { "device":"sopermodule", "uploaddate":datetime.datetime.now().isoformat()[:10] }
measurement = "hanglog"

In [170]:

def uploadfd(fd):
    tags = { "device":"sopermodule", "uploaddate":datetime.datetime.now().isoformat()[:10],
             "fname":os.path.split(fd.fname)[1] }
    measurement = "hanglog"
    rectypespresent = [k for k in fd.reccounts  if fd.reccounts[k] != 0]
    fd.LoadC("".join(rectypespresent))
    for k in rectypespresent:
        fieldkeys = None
        precs = None
        if k in "UZSXYQVGFW":
            fieldkeys = hacktrack.loaders.__getattribute__("recargs"+k)[2]
            precs = fd.__getattribute__("p"+k)
            measurement = "p"+k
        elif k[0] == "a" and k[1] in "ZQFV":
            fieldkeys = hacktrack.loaders.__getattribute__("recargsA"+k[1])[2]
            precs = fd.__getattribute__(k)
            measurement = k
        else:
            print("skipping", k)
            continue
        if k == "Z" or k == "Y":
            precs = precs[~precs["bad"]]
            precs.loc[:,"s"] = precs["s"].astype(int)
            if len(precs) == 0:
                print("skipping all bad Z")
                continue
            if (precs["mx"] == 0).all() and (precs["mx"] == 0).all() and (precs["mz"] == 0).all():
                fieldkeys.remove("mx"); fieldkeys.remove("my"); fieldkeys.remove("mz")
            fieldkeys.extend(["pitch", "roll", "heading"])

        #print(fieldkeys)
        #print(precs.head())

        tags["logchannel"] = k
        records = [ ]
        for idx, row in precs.iterrows():
            #print(row)
            fields = dict((f, row[f])  for f in fieldkeys)
            record = { "measurement":measurement, "tags":tags, "time":idx.isoformat(), "fields":fields }
            #print(record)
            records.append(record)
        print("writing %d records to channel %s" % (len(records), k))
        client.write_points(records)        
        print("  done")
    
    index_fields = { "ft0":fd.ft0.isoformat(), "ft0":fd.ft0.isoformat(), "rectypespresent":" ".join(rectypespresent) }
    record = { "measurement":measurement, "tags":tags, "time":idx.isoformat(), "fields":index_fields }
    measurement = "hanglog_index"
    client.write_points([record])

    print("all done")
    

In [171]:
fd = loaders.FlyDat(fLog)
fd.LoadC("Z")

Extracting knowndate from filename: 2021-03-28
Error: Should be same midnight stamps 2021-03-28 00:00:00 2021-03-27 00:00:00
End found End(8811,3247)
Z:8142, U:29, S:345, X:295, aZ:2485, aQ:72, aF:617, aV:72
linAdifftime 48100760.63192183 count 614
Made for Q 0 last index 0
Made for V 0 last index 0
Loading aQV as no pQV records
Made for aQ 72 last index 72
pCattrname aQ
Setting aRdatetime0 2021-03-28 20:24:23.918916666 from 2021-03-27 21:24:23.963000 with std 14.95
Made for aV 72 last index 72
pCattrname aV
BAD VALUES 8142 [(0, 'Zt000244AAx0000yFFF9zFFFEaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0000n0008oFFF7\n'), (0, 'Zt000244B5xFFFFyFFFBzFFFBaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0002n0002o0000\n'), (0, 'Zt000244C1xFFF6y0002z0008aFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0001nFFF9o0007\n')]
Made for Z 8142 last index 0
Warning, using guessed (not GPS) timing corrected value on Z 8142


0

In [172]:
logfiles = os.listdir(fdir)
for logfile in logfiles:
    print(logfile)
    fLog = os.path.join(fdir, logfile)
    fd = loaders.FlyDat(fLog)
    uploadfd(fd)


hdata-2021-03-28_10-48-33.log
Extracting knowndate from filename: 2021-03-28
Error: Should be same midnight stamps 2021-03-28 00:00:00 2021-03-27 00:00:00
End found End(8811,3247)
Z:8142, U:29, S:345, X:295, aZ:2485, aQ:72, aF:617, aV:72
linAdifftime 48100760.63192183 count 614
Made for Q 0 last index 0
Made for V 0 last index 0
Loading aQV as no pQV records
Made for aQ 72 last index 72
pCattrname aQ
Setting aRdatetime0 2021-03-28 20:24:23.918916666 from 2021-03-27 21:24:23.963000 with std 14.95
Made for aV 72 last index 72
pCattrname aV
BAD VALUES 8142 [(0, 'Zt000244AAx0000yFFF9zFFFEaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0000n0008oFFF7\n'), (0, 'Zt000244B5xFFFFyFFFBzFFFBaFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0002n0002o0000\n'), (0, 'Zt000244C1xFFF6y0002z0008aFEE7b0145c0371w3E4Cx0A7Dy09E6z02ADs31m0001nFFF9o0007\n')]
Made for Z 8142 last index 0
Warning, using guessed (not GPS) timing corrected value on Z 8142
Made for U 29 last index 29
Warning, using guessed (not GPS) timing corrected

Made for U 10 last index 10
Warning, using guessed (not GPS) timing corrected value on U 10
pCattrname pU
Made for S 96 last index 96
Warning, using guessed (not GPS) timing corrected value on S 96
pCattrname pS
Made for X 82 last index 82
Warning, using guessed (not GPS) timing corrected value on X 82
pCattrname pX
Made for aZ 519 last index 519
pCattrname aZ
Made for aA 519 last index 519
pCattrname aA
Made for aF 171 last index 171
pCattrname aF
skipping all bad Z
writing 10 records to channel U
  done
writing 96 records to channel S
  done
writing 82 records to channel X
  done
writing 519 records to channel aZ
  done
skipping aA
writing 171 records to channel aF
  done
all done
hdata-2021-03-30_17-03-31.log
Extracting knowndate from filename: 2021-03-30
End found End(23206,11708)

Z:21470, U:78, S:894, X:764, aZ:5067, aA:5126, aF:1507
linAdifftime 6823479.271628499 count 1572
Made for Q 0 last index 0
Made for V 0 last index 0
Loading aQV as no pQV records
Made for aQ 0 last index

In [110]:
k = client.query("SELECT * FROM pZ where uploaddate='2021-04-01'")

In [106]:
k

ResultSet({'('pZ', None)': [{'time': '2021-03-28T10:48:33.373632Z', 'ax': 0, 'ay': -0.07, 'az': -0.02, 'device': 'sopermodule', 'gx': -2.81, 'gy': 3.25, 'gz': 8.81, 'heading': 82.07358226381176, 'logchannel': 'Z', 'mx': 0, 'my': 0.008888888888888889, 'mz': -0.01, 'pitch': 19.38813543334755, 'q0': 15948, 'q1': 2685, 'q2': 2534, 'q3': 685, 'roll': -16.70165831340808, 's': 49, 'uploaddate': '2021-04-01'}]})

In [143]:
fd = { }
for kk in ["pZ", "pS", "pX", "aQ", "aF"]:
    print("fetching", kk)
    k = client.query("SELECT * FROM %s where uploaddate='2021-04-01'" % kk)
    pK = pandas.DataFrame(k.get_points())
    ds = pandas.DatetimeIndex(pK.time)
    pK = pK.set_index(ds)
    pK.index.name = ""
    pK = pK.drop(columns=["time", "logchannel", "uploaddate", "device"])
    fd[kk] = pK
    print("   ", len(pK), "records")

fetching pZ
    8122 records
fetching pS
    345 records
fetching pX
    295 records
fetching aQ
    72 records
fetching aF
    617 records


In [144]:
fd["aQ"]

,alt,lat,lng,u
,,,,
2021-03-29 09:48:34+00:00,47.0,51.061957,-3.140532,121714000
2021-03-29 09:48:36+00:00,47.1,51.061985,-3.140537,121716000
2021-03-29 09:48:39+00:00,47.3,51.061988,-3.140555,121719000
2021-03-29 09:48:41+00:00,47.4,51.061978,-3.140560,121721000
2021-03-29 09:48:42+00:00,47.5,51.061992,-3.140580,121722000
...,...,...,...,...
2021-03-29 09:50:29+00:00,46.0,51.061928,-3.140453,121829000
2021-03-29 09:50:30+00:00,46.0,51.061923,-3.140468,121830000
2021-03-29 09:50:31+00:00,46.0,51.061902,-3.140488,121831000
